In [2]:
import pandas as pd
from utils import persistence as ps
from urllib3.response import HTTPResponse
from typing import List, Dict, Tuple

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
INPUT_BUCKET: str = 'dq-data'

In [5]:
#load train_set
train_data: str = 'train.csv'
filestream: HTTPResponse = ps.get_file_stream(bucket=INPUT_BUCKET, filename=train_data)
dtypes: Dict[str, str] = {
    'id': 'int64',
    'qid1': 'int64',
    'qid2': 'int64',
    'question1': 'object',
    'question2': 'object',
    'is_duplicate': 'int64'
}
train_df: pd.DataFrame = pd.read_csv(filestream, header=0, usecols=dtypes.keys(), encoding='utf-8', dtype=dtypes, index_col=0)

In [6]:
train_df.head()

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [8]:
# NER question1
from spacy.pipeline import EntityRecognizer
import en_core_web_sm
nlp = en_core_web_sm.load()
#ner = EntityRecognizer(nlp.vocab)
#q1_ner = [nlp(q) for _,q in train_df['question1'].items()]

In [10]:
x_df = train_df.loc[:10]

In [12]:
# similarity between question1 and question2
compute_spacy_similarity = lambda row: nlp(row['question1']).similarity(nlp(row['question2']))
x_df['spacy_similarity'] = x_df.apply(compute_spacy_similarity, axis=1)
x_df.head(10)

,qid1,qid2,question1,question2,is_duplicate,spacy_similarity
id,,,,,,
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.979388
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.816179
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.641907
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.345117
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.692752
5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1,0.781860
6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0,0.316006
7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1,0.862197
8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0,0.582757


In [ ]:
# NER

In [ ]:
# POS

In [ ]:
# unicode
# spell-check
# n-gram similarity
# seq2seq ?



